# Use cases

In [2]:
from agentic_supply.causality_assistant.causal_analysis import CausalAnalysis
from agentic_supply.causality_assistant.causal_graph import CausalGraph

import os

os.chdir("..")

c:\ProjetsCode\AgenticSupply\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Finding Root Causes of Changes in a Supply Chain
Reproduce [reference notebook](https://www.pywhy.org/dowhy/v0.13/example_notebooks/gcm_supply_chain_dist_change.html)  

In [2]:
data_name = "supply_chain_logistics"
causal_graph = CausalGraph(data_name)
causal_analysis = CausalAnalysis(causal_graph)
causal_analysis = CausalAnalysis(causal_graph, model_from_file=True)

2025-09-11 10:59:51,289 : agentic_supply.causality_assistant.causal_graph - INFO - Causal graph instanciated for supply_chain_logistics with form : [('demand', 'submitted'), ('constraint', 'submitted'), ('submitted', 'confirmed'), ('confirmed', 'received')]
2025-09-11 10:59:51,356 : agentic_supply.causality_assistant.causal_analysis - INFO - Causal model instanciated for supply_chain_logistics with causal graph of form : [('demand', 'submitted'), ('constraint', 'submitted'), ('submitted', 'confirmed'), ('confirmed', 'received')]
2025-09-11 10:59:51,356 : agentic_supply.causality_assistant.causal_analysis - INFO - Loading model from supply_chain_logistics_model.pkl
2025-09-11 10:59:51,375 : agentic_supply.causality_assistant.causal_analysis - INFO - Causal model instanciated for supply_chain_logistics with causal graph of form : [('demand', 'submitted'), ('constraint', 'submitted'), ('submitted', 'confirmed'), ('confirmed', 'received')]


In [3]:
causal_graph.visualise()

<Figure size 640x480 with 0 Axes>

In [ ]:
causal_graph.refutate()

In [ ]:
print(causal_analysis.fit_report)

**Why did the average value of received quantity change week-over-week ?**

In [4]:
data_old = causal_analysis.data[causal_analysis.data.week == "w1"]
data_new = causal_analysis.data[causal_analysis.data.week == "w2"]
_, interpretation = causal_analysis.get_distribution_change_attribution(data_new=data_new, data_old=data_old, bootstrap=True)
print(interpretation)

2025-09-11 10:59:57,577 : agentic_supply.causality_assistant.causal_analysis - INFO - Calculating distribution change attribution from data_new with 1000 samples using the bootstrap method
Estimating bootstrap interval...: 100%|██████████| 5/5 [06:50<00:00, 82.03s/it]


Distribution change likelihood scores : {'confirmed': np.float64(-0.42990724435310695), 'constraint': np.float64(0.1108978903850935), 'demand': np.float64(1.8907914881584842), 'received': np.float64(1.014340639760372), 'submitted': np.float64(1.050259978177752)}
        The node demand has the highest likelihood of causing the distribution change seen in your given data.


<Figure size 640x480 with 0 Axes>

## Causal Attributions and Root-Cause Analysis in an Online Shop
Reproduce [reference notebook](https://www.pywhy.org/dowhy/v0.13/example_notebooks/gcm_online_shop.html)  

In [3]:
from dowhy.datasets import sales_dataset

data_2021 = sales_dataset(start_date="2021-01-01", end_date="2021-12-31")
data_2022 = sales_dataset(start_date="2022-01-01", end_date="2022-12-31", change_of_price=0.9)


data_name = "online_shop_data"
causal_graph = CausalGraph(data_name)
causal_analysis = CausalAnalysis(causal_graph)
causal_analysis = CausalAnalysis(causal_graph, model_from_file=True)

2025-09-11 16:54:56,061 : agentic_supply.causality_assistant.causal_graph - INFO - Causal graph instanciated for online_shop_data with form : [('Page Views', 'Sold Units'), ('Revenue', 'Profit'), ('Unit Price', 'Sold Units'), ('Unit Price', 'Revenue'), ('Shopping Event?', 'Page Views'), ('Shopping Event?', 'Sold Units'), ('Shopping Event?', 'Unit Price'), ('Shopping Event?', 'Ad Spend'), ('Ad Spend', 'Page Views'), ('Ad Spend', 'Operational Cost'), ('Sold Units', 'Revenue'), ('Sold Units', 'Operational Cost'), ('Operational Cost', 'Profit')]
2025-09-11 16:54:56,129 : agentic_supply.causality_assistant.causal_analysis - INFO - Causal model instanciated for online_shop_data with causal graph of form : [('Page Views', 'Sold Units'), ('Revenue', 'Profit'), ('Unit Price', 'Sold Units'), ('Unit Price', 'Revenue'), ('Shopping Event?', 'Page Views'), ('Shopping Event?', 'Sold Units'), ('Shopping Event?', 'Unit Price'), ('Shopping Event?', 'Ad Spend'), ('Ad Spend', 'Page Views'), ('Ad Spend', '

In [6]:
causal_graph.visualise()

<Figure size 640x480 with 0 Axes>

In [ ]:
causal_analysis.fit().save_model()
# print(causal_analysis.fit_report)

**What are the key factors influencing the variance in profit ?**

In [9]:
_, _, interpretation = causal_analysis.get_arrow_strength()
print(interpretation)

2025-09-11 11:17:04,652 : agentic_supply.causality_assistant.causal_analysis - INFO - Calculating arrow strength of parent nodes to Profit


Arrow strength (direct effect) scores : {('Operational Cost', 'Profit'): np.float64(212710217004.89752), ('Revenue', 'Profit'): np.float64(646060358690.2659)} (percentages : {('Operational Cost', 'Profit'): 24.769155234820598, ('Revenue', 'Profit'): 75.2308447651794}). 
        The scores indicate how much variance each node is contributing to Profit — where influences through paths over other nodes are ignored. 
        Removing the most impactful causal link, ('Revenue', 'Profit'), increases the variance of Profit by 646060358690.2659 units (75.2308447651794 %).
        


In [10]:
_, _, interpretation = causal_analysis.get_intrinsic_causal_influence()
print(interpretation)

2025-09-11 11:19:18,280 : agentic_supply.causality_assistant.causal_analysis - INFO - Calculating intrinsic causal influence of parent nodes to Profit
Evaluating set functions...: 100%|██████████| 121/121 [01:37<00:00,  1.25it/s]


Intrinsic causal influence scores : {'Shopping Event?': 48024208290.75835, 'Unit Price': 31570947870.42923, 'Ad Spend': -55648190.204112776, 'Page Views': 1709729723.745198, 'Sold Units': 429968056.7559687, 'Revenue': 1340578.9316697493, 'Operational Cost': -3418597.016386006, 'Profit': -8783975.988315068} (percentages : {'Shopping Event?': 58.706397861050064, 'Unit Price': 38.593382223201715, 'Ad Spend': 0.06802620825294706, 'Page Views': 2.090030777589397, 'Sold Units': 0.5256073281756929, 'Revenue': 0.00163876850712999, 'Operational Cost': 0.004179007290562174, 'Profit': 0.010737825932492678}).
        The scores indicate how much variance each node is contributing to Profit — without inheriting the variance from its parents in the causal graph (hence, intrinsic to the node itself).
        The most impactful node Shopping Event? contributes 58.706397861050064 % of the variance in Profit.
        


<Figure size 640x480 with 0 Axes>

**What are the key factors explaining the Profit drop on a particular day ?**

In [5]:
first_day_2022 = data_2022.iloc[[0]]

In [6]:
_, interpretation = causal_analysis.get_anomaly_attribution(first_day_2022, bootstrap=True)

2025-09-11 16:59:02,985 : agentic_supply.causality_assistant.causal_analysis - INFO - Calculating anomaly attribution from anomalous_data with 1 samples using the bootstrap method
Fitting causal mechanism of node Operational Cost: 100%|██████████| 8/8 [00:00<00:00, 176.90it/s]
Evaluating set functions...: 100%|██████████| 119/119 [01:22<00:00,  1.44it/s]
Fitting causal mechanism of node Operational Cost: 100%|██████████| 8/8 [00:00<00:00, 123.04it/s]
Evaluating set functions...: 100%|██████████| 114/114 [00:04<00:00, 24.21it/s]
Fitting causal mechanism of node Operational Cost: 100%|██████████| 8/8 [00:00<00:00, 138.78it/s]
Evaluating set functions...: 100%|██████████| 116/116 [00:04<00:00, 26.43it/s]
Fitting causal mechanism of node Operational Cost: 100%|██████████| 8/8 [00:00<00:00, 183.12it/s]
Evaluating set functions...: 100%|██████████| 108/108 [00:03<00:00, 27.65it/s]
Fitting causal mechanism of node Operational Cost: 100%|██████████| 8/8 [00:00<00:00, 185.34it/s]
Evaluating set

<Figure size 640x480 with 0 Axes>

**What caused the profit drop in Q1 2022 ?**

In [12]:
data_first_quarter_2021 = data_2021[data_2021.Date <= "2021-03-31"]
data_first_quarter_2022 = data_2022[data_2022.Date <= "2022-03-31"]

In [ ]:
_, interpretation = causal_analysis.get_distribution_change_attribution(
    data_old=data_first_quarter_2021, data_new=data_first_quarter_2022, bootstrap=True
)